In [1]:
import keras
import anago

Using TensorFlow backend.
/anaconda3/envs/ner-anago/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# 챗봇 4단계 모델 불러오기
weights = './mecab_model_weights'
params = './mecab_model_params'
preprocessor = './mecab_model_preprocessor'
model = anago.Sequence.load(weights, params, preprocessor)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








In [3]:
# 형태소 분석기 불러오기
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Mecab
mecab = Mecab()

In [33]:
# 챗봇 3단계 모델 불러오기
import pickle
with open('./classifier_tfidf_3.pickle', "rb") as f:
    classifier_tfidf = pickle.load(f)
with open("./mlb_3.pickle", "rb") as f:
    mlb = pickle.load(f)
with open("./tfidf_vectorizer_3.pickle", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

In [ ]:
# 챗봇 3단계 모델 실행에 필요한 함수 정의
QUESTION_RE = re.compile('[^ ㄱ-ㅣ가-힣a-zA-Z]+')

STOPWORDS = set(['은','는','이','가','하','아','것','들','의','그','수','한','나','같','그렇'
                ,'문제','그리고','크','중','나오','지금','생각하','집','어떤','명','생각','이런'
                ,'인','지','을','를','에','스러운','스러워','주','할','만','게','도','져','된','로','고','던','로운','면서'
                ,'사실','이렇','점','싶','말','좀','식당','가게','집','음식점'
                ,'는지','나요','해요','해','는가요','삼','게요','예','는가','습니까','죠','려고요','는지요','서요','였어요','겠'
                ,'인가요','요' '라는','데','해서','세요','어요','을까요','건가요','겠죠','실래요','네요','으세요','지요','인데요'
                ,'드려요','려구요','합니다'])

def text_prepare(text):
    
    # 한글과 스페이스바를 제외한 다른 문자들은 모두 지운다.
    text = QUESTION_RE.sub('', text) 
    
    # mecab 토크나이저
    mecab = Mecab()
    
    # mecab으로 text를 형태소 단위로 나누고 불용어를 지운다.
    text = ' '.join(token for token in mecab.morphs(text) if token not in STOPWORDS)

    return text

#### 테스트용 질문 제작 (분석 잘된거/안된거, 각각 3개씩, 총 102개(=51*2))

***추천 요청 질문 말뭉치*** 
- `음식메뉴` : train 메뉴리스트에 있는 것/없는 것 둘 다 사용해볼 것
- `형용사`,`장소`,`어미` : 실제로 사용자들이 챗봇에게 물어볼만한 말투

- 1) 음식메뉴 + 추천요청어미
- 2) 음식메뉴 + 형용사 + 추천요청어미
- 3) 음식메뉴 + 형용사 + 장소 + 추천요청어미
- 4) 음식메뉴 + 장소 + 형용사 + 추천요청어미
- 5) 형용사 + 추천요청어미
- 6) 형용사 + 음식메뉴 + 추천요청어미
- 7) 형용사 + 형용사 + 추천요청어미
- 8) 형용사 + 음식메뉴 + 장소 + 추천요청어미
- 9) 장소 + 추천요청어미
- 10) 장소 + 음식메뉴 + 추천요청어미
- 11) 장소 + 형용사 + 추천요청어미
- 12) 장소 + 형용사 + 음식메뉴 + 추천요청어미

***문의(장소,리뷰) 질문 말뭉치***
- `가게명` : 골목식당 가게리스트 사용
- `형용사`,`장소`,`어미` : 실제로 사용자들이 챗봇에게 물어볼만한 말투

- 1) 가게명 + 장소문의어미
- 2) 가게명 + 리뷰문의어미
- 3) 가게명 + 형용사 + 리뷰문의어미
- 4) 장소 + 가게명 + 리뷰문의어미
- 5) 장소 + 가게명 + 형용사 + 리뷰문의어미

1. 먼저 아래 셀에서 `text`를 바꿔가며 여러 질문을 테스트한다.
2. 형용사/음식메뉴/장소/가게명의 분석이 잘 된 것은 `X_good`, 뭔가 하나라도 실패한 것은 `X_bad` list에 저장한다.
    - 형용사 : `ADJ`, 음식메뉴 :  `FOD`, 장소 : `LOC`, 가게명 : `ORG`
    
    - `X_good`,`X_bad` list 안의 질문의 개수는 각각 51개씩, (더 많아도 상관 없음)
3. `y_good`,`y_bad` list 안에 각각의 tag를 단다. 
    - (1) : recommendation/questionLOC/questionREV
    - (2) : taste/sanitation/price/atmosphere/prefer
    - (3) : korean/chinese/western/japanese/chicken/asian/etc
    - 태그를 달 때 (1)은 반드시 포함해야 한다. (2),(3)은 포함할 수도 있고 포함하지 않을 수도 있다.
    - (1),(2),(3) 각 태그 종류에서 1개 태그만 선택해야 한다. (korean,chinese를 동시에 달 수 없음)
4. `X_good`, `X_bad` 각각에서 챗봇 4단계, 챗봇 3단계 모델을 실행하고 결과를 저장한다.
5. `okt`/`mecab` 비교

In [4]:
# 1. text를 바꿔가며 테스트
text = '맛있는 제육덮밥 집 알려줘'
text_prepared = ' '.join(word for word in mecab.morphs(text))
model.analyze(text_prepared)

{'words': ['맛있는', '제육', '덮', '밥', '집', '알려줘'],
 'entities': [{'text': '맛있는',
   'type': 'ADJ',
   'score': 1.0,
   'beginOffset': 0,
   'endOffset': 1},
  {'text': '제육 덮 밥 집',
   'type': 'FOD',
   'score': 1.0,
   'beginOffset': 1,
   'endOffset': 5}]}

In [ ]:
# 2. 1번에서 형용사/음식메뉴/장소/가게명의 분석이 잘 되는 질문은 X_good, 잘 안되면 X_bad에 담는다.
X_good = [
    '맛있는 제육덮밥 집 알려줘'
    , '다음 질문'
]
X_bad = [
    
]

In [27]:
ner_anal['entities']

[{'text': '맛있는',
  'type': 'ADJ',
  'score': 1.0,
  'beginOffset': 0,
  'endOffset': 1},
 {'text': '제육 덮 밥 집',
  'type': 'FOD',
  'score': 1.0,
  'beginOffset': 1,
  'endOffset': 5}]

In [ ]:
# 3. X_good과 X_good 각각에 태그 y_good, y_bad를 단다.
y_good = [
    
]
y_bad = [
    
]

In [37]:
# 3. 예시
X_chatbot = [
    '맛있는 파스타집 추천해주세요'
    , '종로에 분위기 좋은 순대국밥집 있나요?'
    , '청결한 한식집 있어요?'
    , '가성비 좋은 짬뽕 집 알려주세요'
    , '줄 서서 기다리는 마라탕 집 있을까요'
    , '보쌈집 위치 알려주십시오'
    , '언니네횟집 주소좀요'
    , '언니네횟집 후기 좀 알려주세요'
    , '언니네횟집 사람들 반응점'
]
y_chatbot = [
    ['recommendation','taste','western']
    ,['recommendation','atmosphere','korean']
    ,['recommendation','sanitation','korean']
    ,['recommendation','price','chinese']
    ,['recommendation','prefer','chinese']
    ,['questionLOC']
    ,['questionLOC']
    ,['questionREV']
    ,['questionREV']
]

In [ ]:
# 챗봇 3단계 모델 실행결과

# SENTENCE 데이터 전처리
X_good_prepared = [text_prepare(x) for x in X_good]
X_bad_prepared = [text_prepare(x) for x in X_bad]

# tf-idf 벡터화
X_good_tfidf = tfidf_vectorizer.transform(X_good)
X_bad_tfidf = tfidf_vectorizer.transform(X_bad)

# tag 데이터 전처리
y_good = mlb.transform(y_good)
y_bad = mlb.transform(y_bad)

# tag 예측
y_good_predicted_labels_tfidf = classifier_tfidf.predict(X_good_tfidf)
y_good_predicted_scores_tfidf = classifier_tfidf.decision_function(X_good_tfidf)
y_bad_predicted_labels_tfidf = classifier_tfidf.predict(X_bad_tfidf)
y_bad_predicted_scores_tfidf = classifier_tfidf.decision_function(X_bad_tfidf)

# 예측된 tag를 텍스트로 변환
y_good_pred_inversed = mlb.inverse_transform(y_good_predicted_labels_tfidf)
y_good_inversed = mlb.inverse_transform(y_chatbot)
y_bad_pred_inversed = mlb.inverse_transform(y_bad_predicted_labels_tfidf)
y_bad_inversed = mlb.inverse_transform(y_bad)

for i in range(0,len(y_good)):
    print('SENTENCE:\t{}\n정답:\t{}\n예측:\t{}\n\n'.format(
        X_good[i],
        y_good_inversed[i],
        y_good_pred_inversed[i]
    ))
for i in range(0,len(y_bad)):
    print('SENTENCE:\t{}\n정답:\t{}\n예측:\t{}\n\n'.format(
        X_bad[i],
        y_bad_inversed[i],
        y_bad_pred_inversed[i]
    ))

In [41]:
# 챗봇 4단계 모델 실행결과 (X_good)
for text in X_good:
    text_prepared = ' '.join(word for word in okt.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

In [ ]:
# 챗봇 4단계 모델 실행결과 (X_bad)
for text in X_bad:
    text_prepared = ' '.join(word for word in okt.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

In [ ]:
q_list = [
    '맛있는 제육덮밥 집 알려줘'
    ,'파스쿠찌 위치 알려주세요'
    ,'담소 순대국밥집 청결한가요?'
    ,'담소 순대국밥집 맛있어요?'
    ,'담소 순대국밥집 맛있는 곳인가요?'
    ,'숯불구이 맛있는곳 알려주실래요?'
    ,'온센텐동 사람들 리뷰 좀 알려주세요'
    ,'온센텐동 위치가 어떻게 되나요'
    ,'파스쿠찌 맛있어요?'
    ,'가성비 대박인 육개장 집 알려주세요'
    ,'맛있는 육개장집 알려주세요'
    ,'육개장 맛있는 곳 알려주세요'
    ,'서울에 맛있는 육개장 가게 알려주세요'
    ,'제주도에 유명한 흑돼지 식당 알려주세요'
    ,'제주도에 맛있는 흑돼지 식당 알려주세요'
    ,'제주도에 흑돼지 식당 맛있는 곳 알려주세요'
    ,'제주도에 흑돼지 식당 맛집 알려주세요'
    ,'제주도에 흑돼지 식당 맛 개쩌는 곳 알려주세요'
    ,'맛있고 깔끔한 한식집 아세요'
    ,'맛있는 깔끔한 한식집 아세요'
]

In [ ]:
# okt 실행결과
for text in q_list:
    text_prepared = ' '.join(word for word in okt.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)

In [ ]:
# mecab 실행결과
for text in q_list:
    text_prepared = ' '.join(word for word in mecab.morphs(text))
    ner = model.analyze(text_prepared)
    display(ner)